In [28]:
import pandas as pd
import requests

In [ ]:
url = 'https://www.asx200list.com/uploads/csv/20170401-asx200.csv'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'}

r = requests.get(url, headers=headers)

with open('20170401-asx200.csv', 'wb') as f:
    f.write(r.content)

In [ ]:
asx = pd.read_csv('20170401-asx200.csv')

asx = asx.drop(0,axis=0)
asx = asx.drop(asx.columns[1:], axis=1)

asx.to_csv('asx200.csv', index=False, header=None)

In [1]:
import eikon as tr
tr.set_app_key('04851d8cdc3d47f7882782edf7d9abbf5999a94a') #CBS02
#A valid app key is required.
df = tr.get_news_headlines(query="APT.AX")
print(df)

                                          versionCreated  \
2020-08-28 14:03:32.951 2020-08-28 14:03:32.951000+00:00   
2020-08-28 07:11:49.317 2020-08-28 07:12:19.375000+00:00   
2020-08-28 07:06:02.174 2020-08-28 07:06:02.174000+00:00   
2020-08-28 07:06:00.908 2020-08-28 07:06:00.908000+00:00   
2020-08-28 05:47:14.000        2020-08-28 05:47:14+00:00   
2020-08-28 03:55:27.000        2020-08-28 03:55:27+00:00   
2020-08-27 12:37:13.073 2020-08-28 02:38:34.895000+00:00   
2020-08-28 02:33:50.000        2020-08-28 02:33:50+00:00   
2020-08-28 01:24:05.064 2020-08-28 01:24:05.064000+00:00   
2020-08-28 00:58:50.557 2020-08-28 00:58:50.557000+00:00   

                                                                      text  \
2020-08-28 14:03:32.951  Australia's BNPL services provider Afterpay ey...   
2020-08-28 07:11:49.317  (EN) AFTERPAY LIMITED Directors Shareholdings ...   
2020-08-28 07:06:02.174     Change of Director's Interest Notice x2-APT.AX   
2020-08-28 07:06:00.908  Au

C:\Users\zoqaz\miniconda3\lib\site-packages\eikon\news_request.py:139: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  headlines_dataframe = pd.DataFrame(headlines, pd.np.array(first_created, dtype='datetime64'), Headline_Selected_Fields)


In [5]:
import os
import pandas as pd
import numpy as np
import cufflinks as cf
from bs4 import BeautifulSoup
from IPython.display import HTML
import configparser as cp
cf.set_config_file(offline=True)
import requests
import glob
from datetime import datetime, timedelta
from dateutil.relativedelta import *
import time

file = open('asx200.csv')

stonks = []
for i in file:
    stonks.append(i.rstrip()+'.AX')

In [9]:
todaydate = datetime.today() + relativedelta(months=-15)


cumcalls = 0
for ticker in stonks[18:199]:
    # Build directories for each ticker
    try:
        os.mkdir('Articles/'+ticker[:-3])
    except FileExistsError:
        pass
    
    # Setup query term
    query = "R:" + ticker + " AND Language:LEN" 
    num = 0
    calls = 0
    # Looping through full historical time window (15 months) dated from today. 
    for i in range(15):
        datestart = todaydate + relativedelta(months=+i)
        dateend = datestart + relativedelta(months=+1)
        dateto = str(dateend.isoformat()[:-15:])
        
        # Eikon query (API capped to 100 results per query)
        headlines = tr.get_news_headlines(query=query, raw_output=True, 
                                  date_from=datestart.isoformat()[:-7:], date_to=dateto, count = 100)["headlines"]
        calls += 1
        cumcalls += 1
        
        # If query returns more than 100 results, create a new query based upon the last articles creation date. 
        # Note that queries are stored as dictionaries in reverse chronological order, so the end indexes represent the 
        # oldest results. 
        maxcall = False
        if len(headlines) == 100:
            maxcall = True
        
        while maxcall == True:
            maxcall = False
            newdate = headlines[99]["firstCreated"][:-5]
            newheadlines = tr.get_news_headlines(query=query, raw_output=True, 
                                  date_from=newdate, date_to=datestart.isoformat()[:-7:], count = 100)["headlines"]
            if len(newheadlines) == 100:
                maxcall = True
            calls +=1
            cumcalls += 1
            headlines = headlines + newheadlines
                
        print("\n\nDownloading articles: ",ticker)
        print(datestart.isoformat()[:-16:] + ': ' + str(len(headlines)))
        print("Cumulative Calls: "+str(cumcalls))
        
        # Once a month of ticker data is retrieved for a stock, the headlines are used to call the stories from the eikon API
        # these are then converted into HTML and parsed. No cleaning is conducted on the text. 
        # export format: Article number; ticker; date published; title; text body
        monthnum = 0
        for i in headlines:
            if i["documentType"] == "Story":
                print(str(monthnum)+' ',end='')
                url = i["storyId"]
                article = tr.get_news_story(url)
                calls += 1
                cumcalls += 1
                raw = BeautifulSoup(article, 'html.parser').get_text()

                mainText = raw.replace(';', ' ')
                title = i['text'].replace(';', ' ')
                date = i['firstCreated']
                export = str(num) + ';' +ticker[:-3] + ';' + date + ';' + title + ';' + mainText 
                outfile = open('Articles/'+ticker[:-3]+'/'+ticker[:-3]+'_'+str(num)+'.txt', 'w',  encoding="utf-8")
                outfile.write(export)
                outfile.close()
                num += 1
                monthnum +=1
            else:
                pass
    print("\n** Calls made for "+ticker+": "+str(calls))
    print("** Cumulative calls made: "+str(cumcalls))
    
        
print("Complete")        



2019-05-29: 33
Cumulative Calls: 1
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 

2019-06-29: 14
Cumulative Calls: 20
0 1 2 3 4 5 6 7 8 9 10 11 

2019-07-29: 54
Cumulative Calls: 33
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 

2019-08-29: 26
Cumulative Calls: 66
0 1 2 3 4 5 6 7 8 9 10 11 12 13 

2019-09-29: 55
Cumulative Calls: 81
0 1 2 3 4 5 6 7 8 9 

2019-10-29: 29
Cumulative Calls: 92
0 1 2 3 

2019-11-29: 44
Cumulative Calls: 97
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 

2019-12-29: 14
Cumulative Calls: 116
0 1 2 3 4 5 6 7 

2020-01-29: 91
Cumulative Calls: 125
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 

2020-02-29: 71
Cumulative Calls: 183
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 

2020-03-29: 13
Cumulative Calls: 209
0 1 2 3 4 5 6 

2020-04-29: 13
Cumulative Calls: 217
0 1 2 3 4 5 6 7 8 

2020

ReadTimeout: HTTPConnectionPool(host='localhost', port=9060): Read timed out. (read timeout=30)